# Imports

In [2]:
# from openai import OpenAI
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import Field, BaseModel
from langchain_core.output_parsers import JsonOutputParser
from langchain.globals import set_debug
import os
from typing import List

set_debug(False)

load_dotenv()

True

In [3]:
# LLM Setup

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.8,
    api_key=os.getenv('OPENAI_API_KEY'),
)
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7fc83047c370>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7fc83047dc90>, root_client=<openai.OpenAI object at 0x7fc830422fb0>, root_async_client=<openai.AsyncOpenAI object at 0x7fc83047c3a0>, temperature=0.8, openai_api_key=SecretStr('**********'), openai_proxy='')

# Testing Template Classes

## Prompt Templates

In [4]:
# Testing PromptTemplate

inputs = {
    'n_days': 7,
    'n_children': 2,
    'activity': 'beach',
    'location': 'Rio de Janeiro'
}

prompt_template = PromptTemplate.from_template(
    "Create a {n_days}-day {location} travel itinerary for a family with {n_children} childrens that likes {activity}."
)

format = prompt_template.format(**inputs)
    
print(format)

Create a 7-day Rio de Janeiro travel itinerary for a family with 2 childrens that likes beach.


In [25]:
# Testing ChatPromptTemplate

inputs = {
    'city': 'Eudoria'
}

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are the narrator of an RPG adventure."),
        ("human", "Tell me about the city we are exploring."),
        ("ai", "You are in {city}, an ancient city known for its mystical ruins and bustling markets."),
        ("human", "I want to know more about the main temple."),
        ("ai", "The Temple of Solara is the spiritual heart of Eldoria, famous for its vast collection of sacred relics and ancient stories.")
    ]
)

dialog = chat_template.format_messages(**inputs)

for msg in dialog:
    print('{:<15} {}'.format(type(msg).__name__ + ':', msg.content))

SystemMessage:  You are the narrator of an RPG adventure.
HumanMessage:   Tell me about the city we are exploring.
AIMessage:      You are in Eudoria, an ancient city known for its mystical ruins and bustling markets.
HumanMessage:   I want to know more about the main temple.
AIMessage:      The Temple of Solara is the spiritual heart of Eldoria, famous for its vast collection of sacred relics and ancient stories.


# Testing Parsers

In [10]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    api_key=os.getenv('OPENAI_API_KEY'),
)

llm.invoke("Suggest words frequently used in the following context: Interview for a data scientist position")

AIMessage(content='Certainly! Here are some words and phrases that are frequently used in the context of an interview for a data scientist position:\n\n1. **Data Analysis**\n2. **Machine Learning**\n3. **Algorithms**\n4. **Statistical Modeling**\n5. **Data Visualization**\n6. **Big Data**\n7. **Predictive Analytics**\n8. **Data Mining**\n9. **Programming Languages** (Python, R, SQL)\n10. **Feature Engineering**\n11. **Data Cleaning**\n12. **A/B Testing**\n13. **Hypothesis Testing**\n14. **Business Intelligence**\n15. **Data Wrangling**\n16. **Neural Networks**\n17. **Deep Learning**\n18. **Model Evaluation**\n19. **Cross-Validation**\n20. **Data Governance**\n21. **ETL (Extract, Transform, Load)**\n22. **Cloud Computing** (AWS, Azure, Google Cloud)\n23. **Collaboration**\n24. **Communication Skills**\n25. **Problem-Solving**\n26. **Critical Thinking**\n27. **Domain Knowledge**\n28. **Data Ethics**\n29. **Data Architecture**\n30. **Visualization Tools** (Tableau, Power BI, Matplotlib)\n

In [9]:
llm.invoke("Suggest a word frequently used in the following context: Interview for a data scientist position")

AIMessage(content='"Analytics" is a word frequently used in the context of an interview for a data scientist position.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 23, 'total_tokens': 43, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_483d39d857', 'finish_reason': 'stop', 'logprobs': None}, id='run-e0a78720-72f1-492c-91f2-1e6943a7528c-0', usage_metadata={'input_tokens': 23, 'output_tokens': 20, 'total_tokens': 43})

In [167]:


class Word(BaseModel):
    word: str = Field(description="The suggested word")
    context: str = Field(description="The context in which the word is to be used")

class Sentences(BaseModel):
    word: str = Field(description="The suggested word")
    sentences: List[str] = Field(description="List of sentences")

word_parser = JsonOutputParser(pydantic_object=Word)
word_template = PromptTemplate(
    template="""
Suggest a word frequently used in the following context: Interview for a data scientist position
    """
)
# word_template.invoke({"context": "Interview for a data scientist position"})
# word_chain = word_template | llm #| word_parser
llm.invoke('Suggest a word frequently used in the following context: Interview for a data scientist position')
# response = word_chain.invoke(
#     {"context": "Interview for a data scientist position"}
# )	
# print(response)

# sentence_parser = JsonOutputParser(pydantic_object=Sentences)
# sentece_template = PromptTemplate(
#     template="""
#         Suggest 3 sentences in the context of {context} using the word '{word}' for a English Speaking Training.
#         ---
#         {output_template}
#     """,
#     input_variables=["context", "word"],
#     partial_variables={"output_template": sentence_parser.get_format_instructions()},
# )
# sentence_chain = sentece_template | llm | sentence_parser

# full_chain = word_chain | sentence_chain

# response = word_chain.invoke(
#     {"context": "Interview for a data scientist position"}
# )	
# print(response)


AIMessage(content='"Algorithm" is a word frequently used in the context of an interview for a data scientist position. It\'s a core concept in data science, encompassing the methods and procedures used for data analysis, machine learning, and problem-solving.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 23, 'total_tokens': 68}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_25624ae3a5', 'finish_reason': 'stop', 'logprobs': None}, id='run-cda4480e-8533-4388-9eac-c85e522e7cca-0', usage_metadata={'input_tokens': 23, 'output_tokens': 45, 'total_tokens': 68})

In [4]:
import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Suggest a word frequently used in the following context: Interview for a data scientist position",
        }
    ],
    model="gpt-4o-2024-08-06",
)
chat_completion

ChatCompletion(id='chatcmpl-A4eIggcpTRuNoK5JlcAAOh9A4SpX8', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='A word frequently used in the context of interviewing for a data scientist position is "algorithm."', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1725671590, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_bbdc8689fd', usage=CompletionUsage(completion_tokens=18, prompt_tokens=23, total_tokens=41))

In [132]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.5,
    api_key=os.getenv("OPENAI_API_KEY"))

# Defina a classe com a estrutura desejada
class Bandeira(BaseModel):
    pais: str = Field(description="nome do pais")
    cores: str = Field(description="cor principal da bandeira")
    historia: str = Field(description="história da bandeira")

# Defina o prompt que será utilizado para pergunta
flag_query = "Me fale da bandeira do Brasil"

# Defina a estrutura que será utilizada para processar a saída
parseador_bandeira = JsonOutputParser(pydantic_object=Bandeira)

prompt = PromptTemplate(
    template="Responda a pergunta do usuário.\n{instrucoes_formato}\n{pergunta}\n",
    input_variables=["pergunta"],
    partial_variables={"instrucoes_formato": parseador_bandeira.get_format_instructions()},
)

chain = prompt | llm | parseador_bandeira

chain.invoke({"pergunta": flag_query})

{'pais': 'Brasil',
 'cores': 'Verde, amarelo, azul e branco',
 'historia': "A bandeira do Brasil foi adotada em 1889 e é composta por um fundo verde com um losango amarelo no centro, contendo uma esfera azul com uma faixa branca contendo a inscrição 'Ordem e Progresso'. As cores representam a família real, a Casa de Bragança, a Casa de Habsburgo e a Casa de Lorena, respectivamente."}